# BUILD FIRST MODEL
(we build here a model with no data preparation!)

In [1]:
#r "nuget:Microsoft.ML,1.5.2"
using Microsoft.ML;
using Microsoft.ML.Data;

MLContext mlContext = new MLContext(seed: 123);

Installed package Microsoft.ML version 1.5.2

In [1]:
public class ModelInput
{
    [LoadColumn(0)]
    public float Temperature { get; set; }

    [LoadColumn(1)]
    public float Luminosity { get; set; }

    [LoadColumn(2)]
    public float Infrared { get; set; }

    [LoadColumn(3)]
    public float Distance { get; set; }

    [LoadColumn(4)]
    public string CreatedAt { get; set; }

    [ColumnName("Label"), LoadColumn(5)]
    public string Source { get; set; }
}

In [1]:
const string TRAIN_DATASET_PATH = "./sensors_data_train.csv";
IDataView trainingData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: TRAIN_DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

const string TEST_DATASET_PATH = "./sensors_data_test.csv";
IDataView testingData = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: TEST_DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');    

In [1]:
var featureColumns = new string[] { "Temperature", "Luminosity", "Infrared", "Distance" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

In [1]:
var model = trainingPipeline.Fit(trainingData);

In [1]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

In [1]:
display(p[@style:"font-size: 40px;"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

MicroAccuracy: 91.93%, MacroAccuracy: 91.82%